In [27]:
import json
from IPython.core.display import display, HTML

In [28]:
import logging
import os

logging.basicConfig(level=os.environ.get("LOGLEVEL", "INFO"),
                    format='%(asctime)s %(name)-12s %(levelname)-8s %(message)s',
                    datefmt='%m-%d %H:%M')

console = logging.StreamHandler()

log_file=logging.FileHandler('logs/ml-lab-local.log', mode='a')


logger=logging.getLogger('ml-lab-local')
logger.addHandler(log_file)

In [29]:
from config import IDENTIFIER
from config import IG_BASE_URL

logger.info('IDENTIFIER: %s, IG_BASE_URL: %s' %(IDENTIFIER,IG_BASE_URL))


03-11 18:17 ml-lab-local INFO     IDENTIFIER: demo2020, IG_BASE_URL: https://demo-api.ig.com/gateway/deal


In [30]:
from secret_manager import SecretManager
sm=SecretManager()
sm.get_secret()

In [31]:
from ig_client import IGClient
ig=IGClient(IG_BASE_URL,IDENTIFIER,sm.secret['ig_password'],sm.secret['ig_api_key'])
ig.login()

In [32]:
session_info=ig.get_session_info()
logger.info(session_info)

03-11 18:17 ml-lab-local INFO     {'clientId': '101811698', 'accountId': 'Z3071N', 'timezoneOffset': 0, 'locale': 'en_GB', 'currency': 'GBP', 'lightstreamerEndpoint': 'https://demo-apd.marketdatasystems.com'}


In [33]:
from lightstreamer_client import LightstreamerClient, LightstreamerSubscription

ls_client=LightstreamerClient(
    lightstreamer_url=session_info["lightstreamerEndpoint"],
    lightstreamer_username=IDENTIFIER,
    lightstreamer_password="CST-%s|XST-%s" %(ig.CST,ig.X_SECURITY_TOKEN)
)

In [34]:
try:
    ls_client.connect()
except Exception as e:
    print(e)
    print("Unable to connect to Lightstreamer Server")

03-11 18:17 root         INFO     Successfully connected to <https://demo-apd.marketdatasystems.com>
03-11 18:17 root         INFO     Started handling of real-time stream


In [35]:
subscription = LightstreamerSubscription(
    mode="MERGE",
    items=["MARKET:CS.D.GBPUSD.CFD.IP"],
    fields=["UPDATE_TIME","MID_OPEN","BID", "OFFER","CHANGE","CHANGE_PCT","HIGH","LOW"]
)

In [36]:
from concurrent.futures import ThreadPoolExecutor
executor = ThreadPoolExecutor(max_workers=2)

In [37]:
def thread_function(item):
     with open('messages.txt', 'a') as f:
        f.write('%s \n' % item)
        
# A simple function acting as a Subscription listener
def on_item_update(item):
    print(item)
    x = executor.submit(thread_function,item)

# Adding the "on_item_update" function to Subscription
subscription.addlistener(on_item_update)

# Registering the Subscription
sub_key = ls_client.subscribe(subscription)
logger.info("Subscription:%s" % sub_key)

03-11 18:17 root         INFO     Successfully subscribed 
03-11 18:17 ml-lab-local INFO     Subscription:1


{'pos': 1, 'name': 'MARKET:CS.D.GBPUSD.CFD.IP', 'values': {'UPDATE_TIME': '18:17:26', 'MID_OPEN': '1.29093', 'BID': '1.28464', 'OFFER': '1.28473', 'CHANGE': '-0.00624', 'CHANGE_PCT': '-0.48', 'HIGH': '1.29775', 'LOW': '1.28249'}}
{'pos': 1, 'name': 'MARKET:CS.D.GBPUSD.CFD.IP', 'values': {'UPDATE_TIME': '18:17:26', 'MID_OPEN': '1.29093', 'BID': '1.28464', 'OFFER': '1.28473', 'CHANGE': '-0.00625', 'CHANGE_PCT': '-0.48', 'HIGH': '1.29775', 'LOW': '1.28249'}}
{'pos': 1, 'name': 'MARKET:CS.D.GBPUSD.CFD.IP', 'values': {'UPDATE_TIME': '18:17:26', 'MID_OPEN': '1.29093', 'BID': '1.28464', 'OFFER': '1.28473', 'CHANGE': '-0.00624', 'CHANGE_PCT': '-0.48', 'HIGH': '1.29775', 'LOW': '1.28249'}}
{'pos': 1, 'name': 'MARKET:CS.D.GBPUSD.CFD.IP', 'values': {'UPDATE_TIME': '18:17:28', 'MID_OPEN': '1.29093', 'BID': '1.28465', 'OFFER': '1.28474', 'CHANGE': '-0.00624', 'CHANGE_PCT': '-0.48', 'HIGH': '1.29775', 'LOW': '1.28249'}}
{'pos': 1, 'name': 'MARKET:CS.D.GBPUSD.CFD.IP', 'values': {'UPDATE_TIME': '18:17

In [38]:
executor.shutdown(wait=False)
subscription.removelisteners()